In [179]:
import pandas as pd
import json
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [200]:
# fetch all listing on a page
def fetch_listings(driver, rollback_page):
    listings = []
    start_page = driver.page_source
    start_html = BeautifulSoup(start_page, "html.parser")
    urls = start_html.findAll('a', {'class':'_3Ajbv _30I97 _1vzK2'})

    print('parsing each listing')
    for url in urls:
        sub_url = BASE_URL + url.get('href')
        driver.get(sub_url)
        sub_html = BeautifulSoup(driver.page_source, "html.parser")

        new_list = parse_list(sub_html, sub_url)
        listings.append(new_list)

    driver.get(rollback_page)

    return listings

In [201]:
# parse info of a list
# TODO: calc commuting time
def parse_list(sub_html, sub_url):
    print('parsing', sub_url)
    try:
        list_name = sub_html.findAll('h1',{'class':'_3Wogd JMF8h lFqTi _1vzK2'})[0].text
        # print('list_name:', list_name)
    except:
        list_name = None

    # price
    try:
        price = sub_html.findAll('p',{'class':'_1zGm8 _3na6W _1vzK2'})[0].text
        # print('price:', price)
    except:
        price = None

    # address
    try:
        location = sub_html.find('p',{'class':'dniCg _3j72o _2rhE-'}).find_all('span', recursive=False)
        address = location[0].text
        # print('address:', address)
        area = location[-1].text.strip('\n')
        # print('area:', area)
    except:
        address = None
        area = None

    # sqft
    try:
        sqft = sub_html.findAll('p',{'class':'_2sIc2 _29qfj _2rhE-'})[-1].text
        # print('sqft:', sqft)
    except:
        sqft = None

    # details
    try:
        overview = sub_html.findAll('tr',{'class':'_2dry3'})
        entry = [tr.findAll('td') for tr in overview]
        details = {}
        for td_list in entry:
            update = {}
            for idx, td in enumerate(td_list):
                update.update({idx: td.text})
            for k,v in update.items():
                if k%2==0:
                    details.update({update[k]:update[k+1]})

        name = details.get("Name", None)
        built_year = details.get("Built year", None)
        # print('details:', details)
    except:
        details = {}

    # amenities
    try:
        amenities = sub_html.findAll('div',{'class':'_3atmT'})
        # print(amenities.find('p').get_text())
        amen = [a.find('p').text for a in amenities]
        # print('amenities:', amen)
    except:
        amen = []

    # description
    try:
        description = sub_html.findAll('div',{'class':'P0YqP'})[0].text
        # print('description:', description)
    except:
        description = None

    # image
    try:
        images = sub_html.findAll('img',{'class':'_3kFcw _2_0I4 _1XbSL'})
        img = [ig['src'] for ig in images]
        # print('images:', img)
    except:
        img = []

    # # agent
    # try:
    #     images = sub_html.findAll('img',{'class':'_3kFcw _2_0I4 _1XbSL'})
    #     img = [ig['src'] for ig in images]
    #     print('images:', img)
    # except:
    #     img = []
    list = {
        'list_name': list_name,
        'name': name,
        'price': price,
        'address': address,
        'area': area,
        'sqft': sqft,
        'built_year': built_year,
        'amenities': amen,
        'description': description,
        'details': details,
        'images': img,
        'link': sub_url,
        'agent': 'TBA'
    }

    return list

In [191]:
# list = [{
#     'list_name': list_name,
#     'name': name,
#     'price': price,
#     'address': address,
#     'area': area,
#     'sqft': sqft,
#     'built_year': built_year,
#     'amenities': amen,
#     'description': description,
#     'details': details,
#     'images': img,
#     'link': link,
#     'agent': 'TBA'
# }]
pd.DataFrame.from_records(list)

list_name          name      price  \
0  1 Bed House for Rent in Kimsia Park  Kimsia Park   $1,500/mo   

            address                         area                 sqft  \
0  Jalan Lada Puteh  Orchard / River Valley (D9)  250 sqft (built-up)   

  built_year              amenities  \
0       None  [Bed, Closet, Aircon]   

                                         description  \
0  This room is a 1 common Bedroom with 1 shared ...   

                                             details  \
0  {'Furnishing': 'Fully', 'Tenure': 'Freehold', ...   

                                              images  \
0  [https://pic2.99.co/v3/Y3Mt9Xt6xXQGTKzvdYgtfj?...   

                                                link agent  
0  https://www.99.co/singapore/rent/property/jala...   TBA

In [231]:
# start page
BASE_URL = 'https://www.99.co'
page_num = 1
rollback_page = f"https://www.99.co/singapore/rent?bathrooms=1&isFilterUnapplied=false&listing_type=rent&map_bounds=0.89031%2C103.594617%2C1.717615%2C104.065088&page_num={page_num}&page_size=35&path=%2Fsingapore%2Frent&price_max=3500&property_segments=residential&query_coords=1.3039947%2C103.8298507&query_limit=commute&query_type=google&radius_max=1000&rental_type=unit&rooms=1&search_commute=1.300693%3A103.856239%3Atransit%3A30&show_cluster_preview=true&show_description=true&show_internal_linking=true&show_meta_description=true&show_nearby=true&sort_field=recency&sort_order=desc"
driver = webdriver.Chrome("../chromedriver")
driver.get(rollback_page)

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_5856/3770966569.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../chromedriver")


In [232]:
# test
page_num = 1
all_listings = []
listings = fetch_listings(driver, rollback_page)
print(listings)
all_listings += listings
page_num += 1
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="appContent"]/div/div[3]/div[1]/div/div[4]/ul/li[7]/a'))).click()
    print('navigate to page:', page_num)
except:
    print('not navigating')
    print('num of listings', len(all_listings))

parsing each listing
parsing https://www.99.co/singapore/rent/property/cardiff-residence-condo-Ygkg3PGJfJcGHAWB9ApwSf#enquiry_position--1+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/tyrwhitt-139-condo-TRDretVX2t4gPzr6REg8wF#enquiry_position--2+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/waterbank-at-dakota-condo-3YAaFzsYCNhYFxEWT2Mz4Q#enquiry_position--3+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/willyn-ville-condo-EBJtaskZiodU3cfRdRx3RS#enquiry_position--4+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/the-glades-condo-JD5QNC8MBcs4UPxoFYm4k3#enquiry_position--5+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/kimsia-court-condo-fRUZnjHV5joJ5Ly6m62vxS#enquiry_position--6+enquiry_source--Search+new_listing_filtere

In [233]:
# start scraping
page_num = 1
all_listings = []

while True:
    rollback_page = f"https://www.99.co/singapore/rent?bathrooms=1&isFilterUnapplied=false&listing_type=rent&map_bounds=0.89031%2C103.594617%2C1.717615%2C104.065088&page_num={page_num}&page_size=35&path=%2Fsingapore%2Frent&price_max=3500&property_segments=residential&query_coords=1.3039947%2C103.8298507&query_limit=commute&query_type=google&radius_max=1000&rental_type=unit&rooms=1&search_commute=1.300693%3A103.856239%3Atransit%3A30&show_cluster_preview=true&show_description=true&show_internal_linking=true&show_meta_description=true&show_nearby=true&sort_field=recency&sort_order=desc&zoom=11"

    print('fetching page', page_num)
    listtings = fetch_listings(driver, rollback_page)
    all_listings += listings
    page_num += 1

    # next p[age
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="appContent"]/div/div[3]/div[1]/div/div[4]/ul/li[7]/a'))).click()
        print('navigate to page:', page_num)
    except:
        print('reached last page', page_num)
        print('total listings', len(all_listings))
        break

fetching page 1
parsing each listing
parsing https://www.99.co/singapore/rent/property/city-loft-condo-7hcELCxz2cSCAaaJYRo4wv#enquiry_position--36+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/gem-residences-condo-xa43fCSqTySvvvVJnsiu83#enquiry_position--37+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/skysuites-17-condo-GScWRV49sors4yKjQ5LR2N#enquiry_position--38+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/the-verve-condo-BbazJBG5TSxj66uDv9DDPE#enquiry_position--39+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/101-bishan-street-12-hdb-n2Y2hN2SWfcF27eo2opcr2#enquiry_position--40+enquiry_source--Search+new_listing_filtered--false
parsing https://www.99.co/singapore/rent/property/skysuites-17-condo-V4HwJYMH9w9aTqWvkKxMVf#enquiry_position--41+enquiry_source--Search+new_l

In [250]:
df = pd.DataFrame(all_listings)
df.to_csv('99co.csv')